In [116]:
import pandas as pd
path_node = r"C:\01 PRZENIESIENIE\ZGC TOMASZÓW MAZOWIECKI\TEST.csv"
dataset_node = pd.read_csv(f"{path_node}",
                     engine = "python",
                     sep=";",
                          decimal = ",")

path_plant = r"C:\01 PRZENIESIENIE\ZGC TOMASZÓW MAZOWIECKI\TEST_PLANT.csv"
dataset_plant = pd.read_csv(f"{path_plant}",
                     engine = "python",
                     sep=";",
                           decimal=",")

In [117]:
dataset_node

,Rok,Miesiąc,Symbol punktu,Adres,"Objętość, m3","Energia, GJ"
0,2022,10,P0283,Warszawska 1,1287.0,146.0
1,2022,10,P0206,ŚW. ANTONIEGO 65/69,893.0,20.0
2,2022,10,P0225,GŁOWACKIEGO 11,410.0,22.0
3,2022,10,P0135,DYWANOWA 7,27.0,2.0
4,2022,10,P0395,DYWANOWA 8,0.0,0.0
...,...,...,...,...,...,...
734,2022,9,P0277,SMUGOWA 13/23,NaN,21.2
735,2022,9,P0386,ZGORZELICKA 6,NaN,7.0
736,2022,9,P0211,AL. MARSZ. J. PIŁSUDSKIEGO 21,NaN,13.0
737,2022,9,P0261,NISKA 33A,NaN,21.0


In [118]:
dataset_plant

,Rok,Miesiąc,Dzień,Temperatura zewnętrzna [°C]\n,Temperatura wody do sieci [°C]\n,Temperatura wody z sieci [°C]\n,Przepływ wody do sieci [t/h]\n,Przepływ wody p. balans [t/h]\n,Moc do sieci\n[MW]\n,Energia wysłana do sieci [GJ]\n,Zużycie wody uzupeł. [m3]\n,Ciśnienie wody do sieci [MPa]\n,Ciśnienie wody z sieci [MPa]\n,Ciśnienie dyspoz. [MPa]\n
0,2022,10,1,12.3,67.1,48.7,446.4,31,10,869,17,0.64,0.3,0.34
1,2022,10,2,12.7,66.8,48.6,446.9,58,10,865,16,0.64,0.3,0.34
2,2022,10,3,11.4,67.4,48.2,459.0,95,11,937,16,0.63,0.3,0.33
3,2022,10,4,11.6,68.6,48.0,437.9,30,11,954,18,0.63,0.3,0.32
4,2022,10,5,12.8,68.0,48.6,426.2,33,10,880,19,0.63,0.3,0.33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,2022,9,26,12.0,68.4,49.7,395.3,38,9,793,24,0.63,0.3,0.33
57,2022,9,27,13.2,68.9,49.5,383.2,53,9,792,26,0.64,0.3,0.34
58,2022,9,28,11.9,69.2,48.7,389.9,99,10,836,23,0.64,0.3,0.34
59,2022,9,29,10.6,69.0,48.4,430.6,97,11,923,27,0.63,0.3,0.33


In [143]:

#Obliczenie średniego schłodzenia w danym roku i miesiącu
def dt_norm(data_plant,month,year):
    """Obliczenie średniego schłodzenia w danym roku i miesiącu
    
    Argumenty:
    data_plant - dataframe z danymi eksploatacyjnymi ze źródła,
    year - rok,
    month - miesiąc roku"""
    
    data = data_plant[(data_plant['Rok'] == year) & (data_plant['Miesiąc'] == month)]
    tsupply_mean = data.iloc[:,4].mean()
    treturn_mean = data.iloc[:,5].mean()
    temp_diff = tsupply_mean - treturn_mean
    
    return temp_diff

def dt_norm_to_df(dataset_node,dataset_plant):
    
    """ Funkcja oblicza oraz zwraca wartość normatywnego schłodzenia
    
    Argumenty:
    dataset_node - dataframe z danymi z węzła,
    dataset_plant - dataframe z danymi ze źródła.
    
    """
    data = dataset_node.copy()
    temp_diff_list = [dt_norm(data_plant=dataset_plant, month=month, year=year) for month, year in zip(data['Miesiąc'],data['Rok'])]
    data['Schłodzenie normatywne, K'] = temp_diff_list
    return data

def dt_real(dataset_node):
    
    """ Funkcja oblicza oraz zwraca wartość rzeczywistego schłodzenia
    
    Argumenty:
    dataset_node - dataframe z danymi z węzła,
    
    
    """
    data = dataset_node.copy()
    data['Schłodzenie rzeczywiste, K'] = data['Energia, GJ']/data['Objętość, m3']/4.19/0.96 * 1000
    return data

def dt_diff(dataset_node):
    
    """ Funkcja oblicza oraz zwraca wartość różnicy pomiędzy schłodzeniem normatywnym, a schłodzeniem rzeczywistym.
    Im większa wartość odchyłki tym gorzej.
    
    
    Argumenty:
    dataset_node - dataframe z danymi z węzła,
    
    
    """
    data = dataset_node.copy()
    data['Odchyłka schłodzenia, K'] =  data['Schłodzenie normatywne, K'] - data['Schłodzenie rzeczywiste, K']
    return data

def run():
    df = dt_norm_to_df(dataset_node,dataset_plant)
    df = dt_real(df)
    df = dt_diff(df)
    return df

In [144]:
run()

,Rok,Miesiąc,Symbol punktu,Adres,"Objętość, m3","Energia, GJ","Schłodzenie normatywne, K","Schłodzenie rzeczywiste, K","Odchyłka schłodzenia, K"
0,2022,10,P0283,Warszawska 1,1287.0,146.0,19.509677,28.202594,-8.692916
1,2022,10,P0206,ŚW. ANTONIEGO 65/69,893.0,20.0,19.509677,5.567924,13.941754
2,2022,10,P0225,GŁOWACKIEGO 11,410.0,22.0,19.509677,13.339931,6.169747
3,2022,10,P0135,DYWANOWA 7,27.0,2.0,19.509677,18.415392,1.094285
4,2022,10,P0395,DYWANOWA 8,0.0,0.0,19.509677,NaN,NaN
...,...,...,...,...,...,...,...,...,...
734,2022,9,P0277,SMUGOWA 13/23,NaN,21.2,15.426667,NaN,NaN
735,2022,9,P0386,ZGORZELICKA 6,NaN,7.0,15.426667,NaN,NaN
736,2022,9,P0211,AL. MARSZ. J. PIŁSUDSKIEGO 21,NaN,13.0,15.426667,NaN,NaN
737,2022,9,P0261,NISKA 33A,NaN,21.0,15.426667,NaN,NaN
